In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

import os
import sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
os.environ['THEANO_FLAGS'] = "device=cpu"

In [ ]:
import copy
import shelve
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.gridspec import GridSpec
from IPython.display import display

plt.rcParams['savefig.dpi'] = 300
pd.set_option("display.precision", 3)

In [ ]:
from experiments.a_data import durations

In [ ]:
from experiments.a_data import cachedir, frame_seqs, dataset, vocabulary, \
    get_ref_pts, detect_invalid_pts, interpolate_positions, pose2d_seqs
from experiments.b_preprocess import skel_feat_seqs, bgr_feat_seqs, bgr_feats
from sltools.transform import Transformation, transform_pose2d, transform_pose3d, transform_frames

cachedir = cachedir.split('.')[0]
tgt_dist = 2
joints = dataset.JointType

flip_mapping = ([joints.ShoulderRight, joints.ElbowRight,
                 joints.WristRight, joints.HandRight, joints.ShoulderLeft,
                 joints.ElbowLeft, joints.WristLeft, joints.HandLeft,
                 joints.HipRight, joints.KneeRight, joints.AnkleRight,
                 joints.FootRight, joints.HipLeft, joints.KneeLeft,
                 joints.AnkleLeft, joints.FootLeft],
                [joints.ShoulderLeft, joints.ElbowLeft,
                 joints.WristLeft, joints.HandLeft, joints.ShoulderRight,
                 joints.ElbowRight, joints.WristRight, joints.HandRight,
                 joints.HipLeft, joints.KneeLeft, joints.AnkleLeft,
                 joints.FootLeft, joints.HipRight, joints.KneeRight,
                 joints.AnkleRight, joints.FootRight])

video = dataset.bgr_frames(0)
poses_2d = dataset.positions(0)
poses_3d = dataset.positions_3d(0)
invalid_masks = detect_invalid_pts(poses_2d)
poses_2d = interpolate_positions(poses_2d, invalid_masks)
poses_3d = interpolate_positions(poses_3d, invalid_masks)
ref2d = get_ref_pts(poses_2d)
ref3d = get_ref_pts(poses_3d)

zshifts = np.mean(tgt_dist - ref3d[2])

In [ ]:
# transformation = Transformation(
#     ref2d=ref2d, ref3d=ref3d, flip_mapping=flip_mapping,
#     frame_width=640,
#     fliplr=False,
#     tilt=5 * np.pi / 180,
#     zshift=zshifts,
#     xscale=1.15, yscale=0.85,
#     zscale=1, tscale=1)

# t = 10
# plt.figure()
# plt.imshow(video[t])
# plt.scatter(poses_2d[t, :, 0], poses_2d[t, :, 1])
# plt.figure()
# plt.imshow(transform_frames(video, transformation)[t])
# trans_pose2d = transform_pose2d(poses_2d, transformation)
# plt.scatter(trans_pose2d[t, :, 0], trans_pose2d[t, :, 1])
# plt.show()

In [ ]:
experiments = []

# for run, directory in enumerate([cachedir + '.run1', cachedir + '.run2', cachedir + '.run3', cachedir + '.run4']):
# for run, directory in enumerate([cachedir + '.run4', cachedir + '.run5', cachedir + '.run6']):
for run, directory in enumerate([cachedir + '.run{}'.format(i) for i in [1, 7]]):
    for report_file in os.listdir(directory):
        if not report_file.endswith(".dat"):
            continue
        
        f = os.path.join(directory, report_file[:-4])
        with shelve.open(f, flag='r') as report:
            if 'analysis' not in report.keys():
                continue
            meta = report['meta']
            name = meta['experiment_name']
            args = report['args']['encoder_kwargs']
            analysis = report['analysis']
            experiments.append((name, run, meta, args, analysis))

In [ ]:
# columns = ['model', 'modality', 'run', 'acc', 'ji', 'acc_filtered', 'ji_filtered']

# analyses = []

# for name, run, meta, args, analysis in experiments:
# #     if meta['modality'] != 'skel':
# #         continue
#     if meta['variant'] != 'tc15':
#         continue
# #     if meta['model'] != 'rnn':
# #         continue

#     model = meta['model']
#     modality = meta['modality']
#     acc = analysis['accuracy'][1]
#     acc_filtered = analysis['accuracy_filtered'][1]
#     ji = analysis['ji'][1]
#     ji_filtered = analysis['ji_filtered'][1]
#     analyses.append((model, modality, run, acc, ji, acc_filtered, ji_filtered))

# analyses = pd.DataFrame(analyses, columns=columns)

# analyses

In [ ]:
columns = ['model', 'modality', 'run', 'acc/val', 'acc/test', 'ji/val', 'ji/test']

analyses = []

for name, run, meta, args, analysis in experiments:
    if meta['modality'] == 'transfer':
        continue
    if meta['variant'] != 'tc15':
        continue
    if meta['model'].endswith("mono"):
        continue

    analyses.append((
        meta['model'], meta['modality'], run, 
        analysis['accuracy'][1], analysis['accuracy'][2], 
        analysis['ji'][1], analysis['ji'][2]))
    analyses.append((
        meta['model'] + ' (filtered)', meta['modality'], run, 
        analysis['accuracy_filtered'][1], analysis['accuracy_filtered'][2], 
        analysis['ji_filtered'][1], analysis['ji_filtered'][2]))

analyses = pd.DataFrame(analyses, columns=columns)

# analyses.groupby(['model', 'modality']).mean()
analyses

# Varying TC size

In [ ]:
dtype = [('model', 'S3'), 
         ('win', 'i'),
         ('nparms', 'i'),
         ('ji', 'f'),
         ('acc', 'f')]

tc_analyses = np.sort([
    np.array((
        m['model'],
        a['filter_dilation'] * (a['tconv_sz'] - 1) + 1,
        a['tconv_sz'] * a['num_tc_filters'],
        r['ji_filtered'][1], 
        r['accuracy_filtered'][1]), 
        dtype=dtype)
    for _, _, m, a, r in experiments 
    if m['modality'] == "skel" and m["model"] in {"hmm", "rnn"}])

plt.figure(dpi=100) 

legend = []
subset = (tc_analyses['model'] == b"rnn")
p1 = plt.scatter(
    tc_analyses[subset]['win'],
    tc_analyses[subset]['ji'],
    s=tc_analyses[subset]['nparms'] / 50,
    marker="o", alpha=0.5)
    
legend = []
subset = (tc_analyses['model'] == b"hmm")
p2 = plt.scatter(
    tc_analyses[subset]['win'],
    tc_analyses[subset]['ji'],
    s=tc_analyses[subset]['nparms'] / 50,
    marker="o", alpha=0.5)

plt.legend([p1, p2], ['rnn', 'hmm'], loc='lower right')

plt.xlabel("window size")
plt.ylabel("Jaccard Index")
plt.xticks(np.arange(1, 32, 4))
plt.gca().set_ylim((.745, .825))
plt.gca().set_axisbelow(True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().grid(color='k', linestyle=':', alpha=.3)

plt.savefig("/home/granger/exp2_variable_tc.pdf", bbox_inches='tight')

# Transfer learning

In [ ]:
source_experiments = {
    (name, run): (meta['modality'], analysis['accuracy_filtered'][2], analysis['ji_filtered'][2])
    for name, run, meta, args, analysis in experiments 
    if meta['modality'] != "transfer"}

columns = [
    'model', 'modality', 'terminate_at', 'run', 
    'acc', 'delta_ref_acc', 'delta_other_acc', 'ji', 'delta_ref_ji', 'delta_other_ji']

transfer_analyses = []

for name, run, meta, args, analysis in experiments:
    if meta['modality'] != 'transfer':
        continue

    model = meta['model']
    terminate_at = args['terminate_at']
    modality, acc_other, ji_other = source_experiments[(args['transfer_from'], run)]
    _, acc_ref, ji_ref = source_experiments[(model + args['transfer_from'][3:], run)]
    acc = analysis['accuracy_filtered'][2]
    ji = analysis['ji_filtered'][2]
    transfer_analyses.append(
        (model, modality, terminate_at, run, 
                  acc, acc - acc_ref, acc - acc_other, 
                  ji, ji - ji_ref, ji - ji_other))

transfer_analyses = pd.DataFrame(transfer_analyses, columns=columns)
# transfer_analyses.sort_values(["modality", "model", "terminate_at"])

In [ ]:
transfer_analyses.groupby(['model', 'modality', 'terminate_at']).mean()

In [ ]:
# compare confusion on video frames

hmm_conf = np.zeros((21, 21))
rnn_conf = np.zeros((21, 21))

for rundir in [cachedir + '.run{}'.format(i) for i in [1, 7]]:
    with shelve.open(os.path.join(rundir, "hmm_bgr_tc15"), flag='r') as report:
        hmm_analysis = report['analysis']
        hmm_conf += hmm_analysis['confusion_filtered'][1]

    with shelve.open(os.path.join(rundir, "rnn_bgr_tc15"), flag='r') as report:
        rnn_analysis = report['analysis']
        rnn_conf += rnn_analysis['confusion_filtered'][1]

hmm_conf /= np.sum(hmm_conf, axis=1, keepdims=True)
rnn_conf /= np.sum(rnn_conf, axis=1, keepdims=True)

conf_diff = hmm_conf - rnn_conf

# plot
plt.figure(dpi=150, figsize=(6, 3))
limits = np.max(abs(conf_diff))
plt.imshow(
    conf_diff, 
    clim=(-limits, limits), 
    cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.5, 0.1), xycoords='axes fraction', xytext=(1.5, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.47, 0.05), xycoords='axes fraction', xytext=(1.47, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.44, 0.92), xycoords='axes fraction', xytext=(1.44, 0.92))
plt.colorbar()

# Compare misclassification
a = np.sum(hmm_conf[1:, 1:]) - np.sum(np.diag(hmm_conf[1:, 1:]))
b = np.sum(rnn_conf[1:, 1:]) - np.sum(np.diag(rnn_conf[1:, 1:]))
print(a, b, (a - b) / b)

plt.savefig("/home/granger/exp1_confdiff_bgr.pdf", bbox_inches='tight')

In [ ]:
# compare confusion on body poses

hmm_conf = np.zeros((21, 21))
rnn_conf = np.zeros((21, 21))

for rundir in [cachedir + '.run{}'.format(i) for i in [1, 7]]:
    with shelve.open(os.path.join(rundir, "hmm_skel_tc15"), flag='r') as report:
        hmm_analysis = report['analysis']
        hmm_conf += hmm_analysis['confusion_filtered'][1]

    with shelve.open(os.path.join(rundir, "rnn_skel_tc15"), flag='r') as report:
        rnn_analysis = report['analysis']
        rnn_conf += rnn_analysis['confusion_filtered'][1]

hmm_conf /= np.sum(hmm_conf, axis=1, keepdims=True)
rnn_conf /= np.sum(rnn_conf, axis=1, keepdims=True)

conf_diff = hmm_conf - rnn_conf

# plot
plt.figure(dpi=150, figsize=(6, 3))
limits = np.max(abs(conf_diff))
plt.imshow(
    conf_diff, 
    clim=(-limits, limits), 
    cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.5, 0.1), xycoords='axes fraction', xytext=(1.5, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.47, 0.05), xycoords='axes fraction', xytext=(1.47, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.44, 0.92), xycoords='axes fraction', xytext=(1.44, 0.92))
plt.colorbar()

# Compare misclassification
a = np.sum(hmm_conf[1:, 1:]) - np.sum(np.diag(hmm_conf[1:, 1:]))
b = np.sum(rnn_conf[1:, 1:]) - np.sum(np.diag(rnn_conf[1:, 1:]))
print(a, b, (a - b) / b)

# plt.tight_layout(rect=[0, 0., 1.2, 1])
plt.savefig("/home/granger/exp1_confdiff_skel.pdf", bbox_inches='tight')

In [ ]:
np.sum(hmm_conf[0, 1:])

In [ ]:
np.sum(rnn_conf[0, 1:])

In [ ]:
# visualize mistaken classes

plt.figure(dpi=150)

plt.imshow(
    np.clip(rnn_conf, 0.0001, 1), 
    clim=(0.001, 1),
    norm=colors.LogNorm(vmin=0.0001, vmax=1., clip=True))
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'])
plt.xticks(np.arange(0, 21), [''] * 21)
plt.colorbar()

plt.tight_layout()
plt.savefig("/home/granger/exp1_rnn_pose_confusion.pdf", bbox_inches='tight')


rnn_conf_bgr = np.zeros((21, 21))

for rundir in [cachedir + '.run{}'.format(i) for i in [1, 7]]:
    with shelve.open(os.path.join(rundir, "rnn_bgr_tc15"), flag='r') as report:
        rnn_analysis = report['analysis']
        rnn_conf_bgr += rnn_analysis['confusion_filtered'][1]

rnn_conf_bgr /= np.sum(rnn_conf_bgr, axis=1, keepdims=True)

plt.figure(dpi=150)

plt.imshow(
    np.clip(rnn_conf_bgr, 0.0001, 1), 
    clim=(0.001, 1),
    norm=colors.LogNorm(vmin=0.0001, vmax=1., clip=True))
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'])
plt.xticks(np.arange(0, 21), [''] * 21)
plt.colorbar()

plt.tight_layout()
plt.savefig("/home/granger/exp1_rnn_bgr_confusion.pdf", bbox_inches='tight')

In [ ]:
# np.random.seed(42)

c1 = 16
c2 = 0

fig = plt.figure(figsize=(8, 5), dpi=150)
all_glosses = np.array([[seq] + list(g) for seq in dataset.default_splits()[0] for g in dataset.glosses(seq)])


p = (all_glosses[:, 1] == c1) / np.sum(all_glosses[:, 1] == c1)
seq1, c1, start1, stop1 = all_glosses[np.random.choice(len(all_glosses), p=p)]
vid1 = dataset.bgr_frames(seq1)

for i, t in enumerate(np.linspace(start1 + 10, stop1 - 10, 5).astype(np.int)):
    frame = vid1[t]
    pose = dataset.positions(seq1)[t]
    x1, x2, y1, y2 = np.min(pose[:, 0]) - 30, np.max(pose[:, 0]) + 30, np.min(pose[:, 1]) - 20, np.max(pose[:, 1]) - 130
    ax = fig.add_subplot(2, 5, i + 1)
    ax.imshow(frame[y1:y2, x1:x2, ::-1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

p = (all_glosses[:, 1] == c2) / np.sum(all_glosses[:, 1] == c2)
seq2, c2, start2, stop2 = all_glosses[np.random.choice(len(all_glosses), p=p)]
vid2 = dataset.bgr_frames(seq2)

for i, t in enumerate(np.linspace(start2 + 10, stop2 - 10, 5).astype(np.int)):
    frame = vid2[t]
    pose = dataset.positions(seq2)[t]
    x1, x2, y1, y2 = np.min(pose[:, 0]) - 30, np.max(pose[:, 0]) + 30, np.min(pose[:, 1]) - 20, np.max(pose[:, 1]) - 130
    ax = fig.add_subplot(2, 5, i + 6)
    ax.imshow(frame[y1:y2, x1:x2, ::-1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.tight_layout(pad=0, h_pad=0)

In [ ]:
t = frame_seqs.sequences[1][0]
frame_seq = frame_seqs.sequences[0][0]
p = pose2d_seqs[0]
plt.imshow(frame_seq[10])

In [ ]:
transformed_frame_seq = transform_frames(frame_seq, t)
plt.imshow(transformed_frame_seq[10])

In [ ]:
hands = bgr_feats(transformed_frame_seq, p)
plt.imshow(np.concatenate(hands[10], axis=1))
plt.colorbar()

In [ ]:
plt.imshow(np.concatenate(bgr_feat_seqs[0][10], axis=1))
plt.colorbar()

In [ ]:
# err vs duration vs time

nbins = 15
rng = np.arange(15, 75, 8)
heatmap_rnn = np.array([
    [0.8237076790416956, 0.8865087277582372, 0.8865087277582372, 0.9311096667253734, 0.9311096667253734, 0.9512303910714499, 0.9311096667253734, 0.9311096667253734, 0.9258279765845283, 0.9522364272887537, 0.9723571516348299, 0.9512303910714499, 0.9052486180805634, 0.8659217477677775, 0.7443440547591771],
    [0.7608766580597115, 0.898502645711337, 0.9519469616540506, 0.9667716280073668, 0.966121371684761, 0.9645016159757861, 0.9627069742631851, 0.971175210936139, 0.9567069529766621, 0.9584158068711331, 0.9566500076146275, 0.9605911212660346, 0.9519924811618184, 0.9166825493866899, 0.7945524618098646],
    [0.6483534727118494, 0.7699439488167763, 0.7873282611367064, 0.7906948252090121, 0.788985678968616, 0.7889909971557523, 0.7906133587819839, 0.7906133587819839, 0.7955576039606042, 0.7932692553364125, 0.7907924528886349, 0.7916054674884142, 0.7882803296378752, 0.7642348953784712, 0.6502263293905041],
    [0.5297797226681489, 0.644312639345597, 0.6600773893037707, 0.666877279512013, 0.666836618940366, 0.6686439248770579, 0.6702196005995311, 0.6711500345166582, 0.6729202941547385, 0.6653461433973238, 0.6645379945422877, 0.6620744156846103, 0.6588860179410522, 0.6360082839431335, 0.5225972957278795],
    [0.4264280902273666, 0.5270087643826037, 0.5310546177383798, 0.5330850070719022, 0.5391263640120293, 0.5391263640120293, 0.5409659519443324, 0.5410829116166375, 0.5409611741799409, 0.537094285261018, 0.5391605422273805, 0.5330472734786716, 0.5314477176248418, 0.5100098436546192, 0.4134741904624286],
    [0.3582215079513415, 0.4709160295499178, 0.467222589779942, 0.467222589779942, 0.4750372605218147, 0.47516326858633084, 0.479275110691594, 0.479275110691594, 0.479275110691594, 0.479275110691594, 0.47537607881635513, 0.4638473737661171, 0.44767155617659243, 0.4267524350156392, 0.30952982325631617],
    [0.2858269903377334, 0.3722162617325532, 0.39198364554925563, 0.391684139733851, 0.38149392234254664, 0.38149392234254664, 0.38149392234254664, 0.38149392234254664, 0.38149392234254664, 0.38149392234254664, 0.3816414062667989, 0.3816414062667989, 0.3816414062667989, 0.3716079614507455, 0.2946922328175119]])
heatmap_hmm = np.array([
    [0.7083483167510551, 0.901150405576342, 0.901150405576342, 0.9414833121064317, 0.9649574905101874, 0.9884316689139433, 0.9884316689139433, 0.9884316689139433, 0.9884316689139433, 0.9884316689139433, 0.9884316689139433, 0.9424498959230568, 0.9224350701261703, 0.9013083095627901, 0.8024423867532052],
    [0.6368393175853759, 0.839090480870381, 0.9305873640382036, 0.9611268216607276, 0.9688913939683073, 0.9787779469743321, 0.9848369757548069, 0.9848369757548069, 0.9828454728339358, 0.9828454728339358, 0.9766440121715088, 0.9744993167182631, 0.9645066245694371, 0.9277641567508252, 0.7890673396025274],
    [0.5316652827245768, 0.7119683956654914, 0.7687047070001396, 0.7878593290431657, 0.7982369078361902, 0.8014875324103441, 0.8065780339310191, 0.8016537169684599, 0.7993134233827618, 0.7945269441644645, 0.7930104007071315, 0.7896822334940211, 0.7825234437087536, 0.751751464593715, 0.5861084912161907],
    [0.4259717536669463, 0.6128686143526147, 0.6383222983382435, 0.6509413473737402, 0.6586186743317092, 0.6577299841456947, 0.6582920221294025, 0.661632515449098, 0.6581949736178113, 0.6532470675696302, 0.6441249663113586, 0.6383403991357146, 0.6333429513927052, 0.6088396475871394, 0.45724573611231123],
    [0.3436729159860165, 0.4996014969648282, 0.5212150042304332, 0.5374009233255292, 0.5433720578745632, 0.5413732748327339, 0.5453229452086105, 0.5434428112565748, 0.5396554243371272, 0.5317409926449393, 0.5279173982123087, 0.520042290153341, 0.5042917063819283, 0.4765662521316257, 0.36047004649474623],
    [0.2660252855357045, 0.4545986567403759, 0.4704408084541456, 0.4704408084541456, 0.4703148003896294, 0.4585873601782835, 0.44678902613753646, 0.4428165685104178, 0.44692155367152575, 0.4555502638037111, 0.44758475401108433, 0.43553039983224157, 0.415924251993777, 0.40344656037651055, 0.213468148985305],
    [0.2155774564289645, 0.31273008488024195, 0.35137954855928916, 0.36111348755993805, 0.36111348755993805, 0.3612702601351889, 0.37100419913583776, 0.37100419913583776, 0.3708474265605869, 0.38163706850432094, 0.3617129121123676, 0.3715943350372688, 0.3715943350372688, 0.38132827403791775, 0.22473449713886828]])

fig = plt.figure(figsize=(4, 3.5), dpi=150)

ax0 = fig.add_subplot(111, frameon=False)
ax0.set_ylabel("gesture duration")
ax0.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)


ax1 = fig.add_subplot(211)
# ax1.imshow(heatmap_rnn, clim=(0.2, 1), norm=colors.LogNorm(vmin=0.2, vmax=1))
ax1.imshow(heatmap_rnn, clim=(0.2, 1))
ax1.invert_yaxis()
ax1.set_yticks(np.arange(-.5, len(rng) - .5, 2))
ax1.set_yticklabels(np.arange(rng[0], rng[-1] + 0.1, rng[1] - rng[0])[::2].astype(np.int))
ax1.set_xticks(np.arange(-.5, nbins + .5, 5))
ax1.set_xticklabels([])
ax1.set_title("BD-RNN")

ax2 = fig.add_subplot(212)
# im = ax2.imshow(heatmap_hmm, clim=(0.2, 1), norm=colors.LogNorm(vmin=0.2, vmax=1))
im = ax2.imshow(heatmap_hmm, clim=(0.2, 1))
ax2.invert_yaxis()
ax2.set_yticks(np.arange(-.5, len(rng) - .5, 2))
ax2.set_yticklabels(np.arange(rng[0], rng[-1] + 0.1, rng[1] - rng[0])[::2].astype(np.int))
ax2.set_xticks(np.arange(-.5, nbins + .5, 5))
ax2.set_xticklabels(["{:0.1f}".format(x) for x in np.arange(0, 1.01, 5 / nbins)])
ax2.set_xlabel(r"relative time (0=beginning, 1=end of gesture)")
ax2.set_title("NN-HMM")

fig.colorbar(im, ax=[ax1, ax2])

plt.savefig("/home/granger/exp1_err_vs_duration_vs_time.pdf", bbox_inches='tight')

In [ ]:
print(np.mean(heatmap_rnn, axis=1))
print(np.mean(heatmap_hmm, axis=1))